# **Calculate Course Similarity using BoW Features**

In [2]:
import pandas as pd 
import numpy as np
from scipy.spatial.distance import cosine, jaccard, euclidean
import nltk as nltk
 

In [3]:
path = '/home/uzokmurod/Desktop/amaliyot/book recommendation sysrem/data/external/df_titles.csv'
bow_path = '/home/uzokmurod/Desktop/amaliyot/book recommendation sysrem/data/processed/bow_df'
df_title = pd.read_csv(path)
bow_df = pd.read_csv(bow_path)


In [4]:
df_title

,id,authors,original_title,title,language_code
0,1,Suzanne Collins,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng
1,2,"J.K. Rowling, Mary GrandPré",Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng
2,3,Stephenie Meyer,Twilight,"Twilight (Twilight, #1)",en-US
3,4,Harper Lee,To Kill a Mockingbird,To Kill a Mockingbird,eng
4,5,F. Scott Fitzgerald,The Great Gatsby,The Great Gatsby,eng
...,...,...,...,...,...
9995,9996,Ilona Andrews,Bayou Moon,"Bayou Moon (The Edge, #2)",eng
9996,9997,Robert A. Caro,Means of Ascent,"Means of Ascent (The Years of Lyndon Johnson, #2)",eng
9997,9998,Patrick O'Brian,The Mauritius Command,The Mauritius Command,eng
9998,9999,Peggy Orenstein,Cinderella Ate My Daughter: Dispatches from th...,Cinderella Ate My Daughter: Dispatches from th...,eng


In [5]:


def pivot_two_bows(basedoc, comparedoc):
    """
    Pivot two bag-of-words (BoW) representations for comparison.

    Parameters:
    basedoc (DataFrame): DataFrame containing the bag-of-words representation for the base document.
    comparedoc (DataFrame): DataFrame containing the bag-of-words representation for the document to compare.

    Returns:
    DataFrame: A DataFrame with pivoted BoW representations for the base and compared documents,
    facilitating direct comparison of word occurrences between the two documents.
    """

    # Create copies of the input DataFrames to avoid modifying the originals
    base = basedoc.copy()
    base['type'] = 'base'  # Add a 'type' column indicating base document
    compare = comparedoc.copy()
    compare['type'] = 'compare'  # Add a 'type' column indicating compared document

    # Concatenate the two DataFrames vertically
    join = pd.concat([base, compare])
    # Pivot the concatenated DataFrame based on 'doc_id' and 'type', with words as columns
    joinT = join.pivot(index=['book_ids', 'type'], columns='words').fillna(0).reset_index()

    # Assign meaningful column names to the pivoted DataFrame
    joinT.columns = ['book_ids', 'type'] + [t[1] for t in joinT.columns][2:]

    # Return the pivoted DataFrame for comparison
    return joinT


In [6]:
bow_df

,doc_indices,book_ids,words,counts
0,0,2767052,Collins,1
1,0,2767052,Games,3
2,0,2767052,Hunger,3
3,0,2767052,Suzanne,1
4,1,3,'s,2
...,...,...,...,...
60669,9998,8565083,Girlie-Girl,2
60670,9998,8565083,Orenstein,1
60671,9999,8914,John,1
60672,9999,8914,War,2


In [7]:
bow_df.columns

Index(['doc_indices', 'book_ids', 'words', 'counts'], dtype='object')

In [17]:
def recommendetion():
    
    courses = bow_df["doc_indices"].unique()
    course1 = bow_df[bow_df['doc_indices'] == 2]
    
    rows = []
    for id in courses:
        course2 = bow_df[bow_df['doc_indices'] == id]
        bow_vectors = pivot_two_bows(course2, course1)
        similarity = 1-euclidean(bow_vectors.iloc[0, 2:], bow_vectors.iloc[1, 2:])
        if similarity > 0.5:
            rows.append({'book_id':id, "similarity":similarity})
    df = pd.DataFrame(rows)
    
    
    return df_title.loc[df_title['id'].isin(df['book_id'])], df


recommended, books =  recommendetion()
        

In [16]:
books


,book_id,similarity
0,0,1.0
1,1,1.0
2,3,1.0
3,4,1.0
4,5,1.0
...,...,...
9994,9995,1.0
9995,9996,1.0
9996,9997,1.0
9997,9998,1.0


In [14]:
recommended

,id,authors,original_title,title,language_code
1,2,"J.K. Rowling, Mary GrandPré",Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng
